# DATA PARSER

In [2]:
%matplotlib notebook

import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import glob
from IPython.display import clear_output
import numpy as np

In [103]:
df9 = df1['StudioEventData']
df9.unique()

array([nan, 'IE_O', 'V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png',
       'VA_950x600.png', 'VB_950x600.png', 'VC_950x600.png',
       'VD_950x600.png', 'IE_uvod', 'IE_1', 'https://www.alza.sk/',
       'https://www.alza.sk/muj-ucet/profil',
       'https://www.alza.sk/muj-ucet/profil?layoutAutoChange=1',
       'https://www.alza.sk/muj-ucet/objednavky.htm',
       'https://www.alza.sk/muj-ucet/objednavka-169446698.htm', 'IE_2',
       'https://www.alza.sk/sony-cybershot-dsc-w800?dq=601733', 'IE_8',
       'IE_3', 'https://www.alza.sk/zlavove-tornado',
       'https://www.alza.sk/vypredaj/u0.htm',
       'https://www.alza.sk/vypredaj/u0.htm#f&cst=0&pg=1&pn=1&prod=',
       'IE_7',
       'https://www.alza.sk/sony-cybershot-dsc-w800?dq=601733#recenze',
       'IE_4', 'https://www.alza.sk/digitalne-fotoaparaty/18843129.htm',
       'https://www.alza.sk/di

## Main work

##### Existuju separatne subory .tsv pre kazdeho testera a v nich ma kazdy tester data pre ulohy 1 az 8. Tieto data chceme spojit a mat ich rozdelene jednotlivo po jednotlivych taskoch v .tsv file

Nacitam si vsetky nazvy tsv suborov z priecinka tzv. vsetkych testerov

In [3]:
path = './data_by_testers'

testers_names = []
file_paths = glob.glob(path + '/*.tsv')
file_names = []
# structure 'path\\name_of_file.format'
for f in file_paths:
    file_names.append(f.replace("\\","/"))
    testers_names.append(f[15:23])


Nacitam si z nazvom jednotlivych testerov a ulozim do do pola dataframov - df_testers[]

In [4]:
missing_columns = ['[Q01]Value', '[Q01_1]Value', '[Q01_1_1]Value', '[Q01_1_1_1]Value', 
                   '[Q01_1_1_1_1]Value', '[Q01_1_1_1_1_1]Value', '[Q01_1_1_1_1_1_1]Value', '[Q01_1_1_1_1_1_1_1]Value']

df_testers = {}

for f in file_names:
    df_testers[f[15:23]] = pd.read_csv(f, sep="\t",low_memory=False)
    
    if(len(df_testers[f[15:23]].columns.values) == 82):
        index = 5
        for col in missing_columns:
            df_testers[f[15:23]].insert(loc = index, column = col, value = np.nan)
            index += 1
            
    name_last_column = df_testers[f[15:23]].iloc[:,-1].name
    df_testers[f[15:23]] = df_testers[f[15:23]].drop(columns=[name_last_column])
    
print("Done")

Done


Trieda Tester, pre kazdeho testera bude mat svoje data rozdelene podla taskov - task 1 az 8

In [5]:
class Tester:

    def __init__(self):
        self.tasks = {}
       
    def assign(self, name, data):
         self.tasks[name] = data
     

Funkcia, ktora pre testera z pola (podla indexu) vyberie data, rozdeli a utriedi podla taskov od 1 po 8 s jednotlivymi indexami odkial pokial su data pre dany task (ulohu) a vrati indexy

In [5]:
def get_user_task(df_testers):
    """
    :param df_testers - all user's data
    @return indexes - indexes of each user's task
    """
    df_temp = df_testers['StudioEventData'] 
    _d = []
    for i in range(1,9):
        _d.append(["IE_"+str(i), df_temp[df_temp == "IE_"+str(i)].index.values])
        
    # sort values based on STARTING POINT of given task
    tester_sorted = sorted(_d, key=lambda x: x[1][0])
    
    #length_of_dataset = len(df_testers[user_index])
    length_of_dataset = len(df_testers)
    indexes = []
    
    # saving indexes
    indexes.append([tester_sorted[0][0], [tester_sorted[0][1][0], tester_sorted[1][1][0]]])

    for j in range(1,len(tester_sorted) - 1):
        indexes.append([tester_sorted[j][0], [tester_sorted[j][1][0], tester_sorted[j+1][1][0]]])
    
    #index pred koncom testu, kde uz zacina IE_OB
    try:
        last_index = df_temp[df_temp == "IE_OB"].index.values[0]
    except:
        print("Didnt find IE_OB. Trying to find IE_dot")
        try:
            last_index = df_temp[df_temp == "IE_dot"].index.values[0]
        except:
            print("Didnt find IE_OB or IE_dot")
            
    indexes.append([tester_sorted[j + 1][0], [tester_sorted[j + 1][1][0], last_index]])
    
    return indexes
 

Pre testerov najde indexy jednotlivych uloh z dat, vytvori instanciu triedy Tester a ulozi ju do pola vsetkych testerov

In [6]:
"""
Get tasks for each user and create instance of class
"""
users = {}
for tester in df_testers:
    
    _user_tasks = get_user_task(df_testers[tester])
    users[tester] = Tester()
    
    for u in _user_tasks:
        start = u[1][0]
        end = u[1][1]
        
        users[tester].assign(u[0], df_testers[tester][start:end])
    
print("Done")

Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Didnt find IE_OB. Trying to find IE_dot
Done


In [121]:
def transform_data_based_on_task(users):
    # Nazvy taskov
    task_names = ["IE_1", "IE_2", "IE_3", "IE_4", "IE_5", "IE_6", "IE_7", "IE_8"]
    total_length = 0

    for task_name in task_names:
        clear_output()
        print("Working on " + task_name)
        df_fin = pd.DataFrame()
        for tester_name in users:
            print("* Working on " + tester_name)
            #print(" * " + tester_name + " - " + str(len(users[tester_name].tasks[task_name].columns.values)))
            df_fin = df_fin.append(users[tester_name].tasks[task_name])
            #print(users[tester_name].tasks[task_name].iloc[:,-1].name)

        total_length += len(df_fin)
        print("Writing....")
        df_fin.to_csv("data_by_tasks/" + task_name + ".tsv", sep="\t", index=False)


    print("Done")
    print(total_length)

In [134]:
transform_data_based_on_task()

## Calibration data

In [66]:
def get_user_calibration_task(df_testers):
    """
    :param df_testers - all user's data
    @return indexes - indexes of each user's task
    """
    calib = ['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png',
       'VA_950x600.png', 'VB_950x600.png', 'VC_950x600.png',
       'VD_950x600.png']
    
    df_temp = df_testers['StudioEventData'] 
    _d = []
    for c in calib:
        ind_values = df_temp[df_temp == c].index.values
        #if(len(ind_values) > 2):
           # _d.append([c, ind_values[0:2], ind_values[2:4]])
        #else:
        _d.append([c, ind_values[0:2]])
      
    # sort values based on STARTING POINT of given task
    tester_sorted = sorted(_d, key=lambda x: x[1][0])
    
    #print(tester_sorted)
    
    #length_of_dataset = len(df_testers[user_index])
    length_of_dataset = len(df_testers)
    indexes = []
    
    # saving indexes
    indexes.append([tester_sorted[0][0], [tester_sorted[0][1][0], tester_sorted[1][1][0]]])

    for j in range(1,len(tester_sorted) - 1):
        indexes.append([tester_sorted[j][0], [tester_sorted[j][1][0], tester_sorted[j+1][1][0]]])
    
    #index pred koncom testu, kde uz zacina IE_OB
    try:
        last_index = df_temp[df_temp == 'IE_uvod'].index.values[0]
    except:
        print("Didnt find IE_uvod.")
    
    indexes.append([tester_sorted[j + 1][0], [tester_sorted[j + 1][1][0], last_index]])
    
    return indexes
 

In [87]:
def get_user_calibration_task_second(df_testers):
    """
    :param df_testers - all user's data
    @return indexes - indexes of each user's task
    """
    calib = ['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png']
    
    df_temp = df_testers['StudioEventData'] 
    _d = []
    for c in calib:
        ind_values = df_temp[df_temp == c].index.values
        if(len(ind_values) > 2):
            _d.append([c, ind_values[2:4]])
      
    if(len(ind_values) > 2):
        # sort values based on STARTING POINT of given task
        tester_sorted = sorted(_d, key=lambda x: x[1][0])

        #print(tester_sorted)

        #length_of_dataset = len(df_testers[user_index])
        length_of_dataset = len(df_testers)
        indexes = []

        # saving indexes
        indexes.append([tester_sorted[0][0], [tester_sorted[0][1][0], tester_sorted[1][1][0]]])

        for j in range(1,len(tester_sorted) - 1):
            indexes.append([tester_sorted[j][0], [tester_sorted[j][1][0], tester_sorted[j+1][1][0]]])

        #index pred koncom testu, kde uz zacina IE_OB
        try:
            last_index = df_temp[df_temp == 'VA_950x600.png'].index.values[3]
        except:
            print("Didnt find VA_950x600.png.")

        indexes.append([tester_sorted[j + 1][0], [tester_sorted[j + 1][1][0], last_index]])

        return indexes
    else:
        return 0

In [32]:
def transform_calibration_data_based_on_task(users):
    # Nazvy taskov
    
    task_names = ['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png',
       'VA_950x600.png', 'VB_950x600.png', 'VC_950x600.png',
       'VD_950x600.png']
    
    total_length = 0

    for task_name in task_names:
        clear_output()
        print("Working on " + task_name)
        df_fin = pd.DataFrame()
        for tester_name in users:
            print("* Working on " + tester_name)
            #print(" * " + tester_name + " - " + str(len(users[tester_name].tasks[task_name].columns.values)))
            df_fin = df_fin.append(users[tester_name].tasks[task_name])
            #print(users[tester_name].tasks[task_name].iloc[:,-1].name)

        total_length += len(df_fin)
        print("Writing....")
        df_fin.to_csv("calibration_data_by_tasks/" + task_name[0:len(task_name)-4] + ".tsv", sep="\t", index=False)


    print("Done")
    print(total_length)

In [25]:
"""
Get tasks for each user and create instance of class
"""
users_calib = {}
for tester in df_testers:
    
    _user_tasks = get_user_calibration_task(df_testers[tester])
    users_calib[tester] = Tester()
    
    for u in _user_tasks:
        start = u[1][0]
        end = u[1][1]
        
        users_calib[tester].assign(u[0], df_testers[tester][start:end])
    
print("Done")

Done


In [33]:
transform_calibration_data_based_on_task(users_calib)
print("Done")

Working on VD_950x600.png
* Working on tester56
* Working on tester22
* Working on tester60
* Working on tester34
* Working on tester46
* Working on tester09
* Working on tester33
* Working on tester41
* Working on tester48
* Working on tester59
* Working on tester44
* Working on tester64
* Working on tester14
* Working on tester31
* Working on tester53
* Working on tester08
* Working on tester12
* Working on tester54
* Working on tester38
* Working on tester61
* Working on tester66
* Working on tester13
* Working on tester69
* Working on tester17
* Working on tester37
* Working on tester58
* Working on tester47
* Working on tester20
* Working on tester07
* Working on tester29
* Working on tester36
* Working on tester49
* Working on tester52
* Working on tester21
* Working on tester50
* Working on tester27
* Working on tester10
* Working on tester35
* Working on tester25
* Working on tester65
* Working on tester68
* Working on tester39
* Working on tester40
* Working on tester43
* Work

In [89]:
"""
Get tasks for each user and create instance of class
"""
users_calib_second = {}
for tester in df_testers:
    
    _user_tasks = get_user_calibration_task_second(df_testers[tester])
    if(_user_tasks != 0):
        users_calib_second[tester] = Tester()

        for u in _user_tasks:
            start = u[1][0]
            end = u[1][1]

            users_calib_second[tester].assign(u[0], df_testers[tester][start:end])
    else:
        continue
    
print("Done")

Done


In [92]:
def transform_calibration_data_based_on_task_second(users):
    # Nazvy taskov
    
    task_names = ['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png']
    
    total_length = 0

    for task_name in task_names:
        clear_output()
        print("Working on " + task_name)
        #df_fin = pd.DataFrame()
        df_fin = pd.read_csv("calibration_data_by_tasks0/" + task_name[0:len(task_name)-4] + ".tsv", low_memory=False, sep="\t")
        for tester_name in users:
            print("* Working on " + tester_name)
            #print(" * " + tester_name + " - " + str(len(users[tester_name].tasks[task_name].columns.values)))
            df_fin = df_fin.append(users[tester_name].tasks[task_name])
            #print(users[tester_name].tasks[task_name].iloc[:,-1].name)

        total_length += len(df_fin)
        print("Writing....")
        df_fin.to_csv("calib_data_full/" + task_name[0:len(task_name)-4] + ".tsv", sep="\t", index=False)


    print("Done")
    print(total_length)

In [93]:
transform_calibration_data_based_on_task_second(users_calib_second)

Working on V9_1710x1080.png
* Working on tester61
* Working on tester32
* Working on tester57
* Working on tester53
* Working on tester60
* Working on tester26
* Working on tester22
* Working on tester66
* Working on tester36
* Working on tester58
* Working on tester40
* Working on tester33
* Working on tester70
* Working on tester52
* Working on tester51
* Working on tester28
* Working on tester21
* Working on tester29
* Working on tester50
* Working on tester44
* Working on tester71
* Working on tester24
* Working on tester48
* Working on tester64
* Working on tester46
* Working on tester38
* Working on tester25
* Working on tester47
* Working on tester65
* Working on tester41
* Working on tester27
* Working on tester42
* Working on tester37
* Working on tester31
* Working on tester63
* Working on tester49
* Working on tester55
* Working on tester30
* Working on tester56
* Working on tester34
* Working on tester54
* Working on tester69
* Working on tester20
* Working on tester43
* Wo

### TESTING

In [94]:
dataset = pd.read_csv("calib_data_full/V1_190x120.tsv", low_memory=False, sep="\t")

In [97]:
len(dataset['ParticipantName'].unique())

58

In [53]:
dd = df_testers["tester08"]
#for v in dd.iloc[90:5014].values:
#for v in dd.iloc[90:91].values:
    #print(v)
#dd.iloc[90:5014].columns.values 
dd[dd['StudioEventData'] == "V1_190x120.png"]


,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,[Q01]Value,[Q01_1]Value,[Q01_1_1]Value,[Q01_1_1_1]Value,[Q01_1_1_1_1]Value,...,CamRightY,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight
789,02/03/2018,3.4.8,DP_oboznamenost,New test,tester08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,02/03/2018,3.4.8,DP_oboznamenost,New test,tester08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
dd[dd['StudioEventData'] == "V2_950x120.png"]

,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,[Q01]Value,[Q01_1]Value,[Q01_1_1]Value,[Q01_1_1_1]Value,[Q01_1_1_1_1]Value,...,CamRightY,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight
1097,02/03/2018,3.4.8,DP_oboznamenost,New test,tester08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401,02/03/2018,3.4.8,DP_oboznamenost,New test,tester08,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
_user_tasks = get_user_task(df_testers["tester38"])

In [128]:
print(dd[dd['StudioEventData'] == 'IE_OB'].iloc[:,0:3])

Empty DataFrame
Columns: [ExportDate, StudioVersionRec, StudioProjectName]
Index: []


In [110]:
dd['StudioEventData'].unique()
dd[dd['StudioEventData'] == 'IE_6'].index.values

array([71820, 72669], dtype=int64)

In [69]:
dd = df_testers["tester58"]

In [130]:
dd['StudioEventData'].unique()

array([nan, 'IE_O', 'V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png',
       'VA_950x600.png', 'VB_950x600.png', 'VC_950x600.png',
       'VD_950x600.png', 'IE_uvod', 'IE_3', 'https://www.alza.sk/',
       'https://www.alza.sk/MailingActionList.htm',
       'https://www.alza.sk/MailingActionList.htm?layoutAutoChange=1',
       'https://www.alza.sk/zlavove-tornado', 'IE_4',
       'https://www.alza.sk/digitalne-fotoaparaty/18843129.htm',
       'https://www.alza.sk/digitalne-fotoaparaty/18843129.htm#f&limit=200---1&cst=0&pg=1&prod=&sc=910',
       'https://www.alza.sk/digitalne-fotoaparaty/18843129.htm#f&limit=200---1&cst=0&pg=1&pn=1&prod=&sc=910',
       'https://www.alza.sk/lacne-digitalne-fotoaparaty/18843129.htm#f&limit=200---1&cst=0&pg=1&pn=1&prod=&sc=910',
       'IE_2', 'https://www.alza.sk/sony-cybershot-dsc-w800?dq=601733',
       'IE_5', 'IE_1', '

In [70]:
print(dd[dd['StudioEventData'] == 'VD_950x600.png'].iloc[:,0:3])

      ExportDate StudioVersionRec StudioProjectName
4911    3/8/2018            3.4.8   DP_oboznamenost
5217    3/8/2018            3.4.8   DP_oboznamenost
90616   3/8/2018            3.4.8   DP_oboznamenost
90916   3/8/2018            3.4.8   DP_oboznamenost


In [18]:
calib = ['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png',
       'VA_950x600.png', 'VB_950x600.png', 'VC_950x600.png',
       'VD_950x600.png']


for c in calib:
    print(c)
    print(dd[dd['StudioEventData'] == c].index.values[0:2])
    print("\n")

IE_O
[ 90 758]


V1_190x120.png
[ 802 1106]


V2_950x120.png
[1111 1415]


V3_1710x120.png
[1419 1723]


V4_190x600.png
[1728 2032]


V5_950x600.png
[2035 2340]


V6_1710x600.png
[2345 2649]


V7_190x1080.png
[2654 2958]


V8_950x1080.png
[2962 3266]


V9_1710x1080.png
[3271 3575]


VA_950x600.png
[3580 3883]


VB_950x600.png
[3887 4192]


VC_950x600.png
[4195 4499]


VD_950x600.png
[4501 4806]


